# Train Stable Diffusion with CodeFlare

In [1]:
!pwd

/opt/app-root/src/teddy-demo


In [1]:
# !pip install codeflare-sdk
# !pip install -Ur 05_dreambooth_finetuning/dreambooth/requirements.txt

## Prep Infra

In [6]:
# Import pieces from codeflare-sdk
from codeflare_sdk.cluster.cluster import Cluster, ClusterConfiguration
from codeflare_sdk.cluster.auth import TokenAuthentication

In [7]:
# Create authentication object for user permissions
# IF unused, SDK will automatically check for default kubeconfig, then in-cluster config
# KubeConfigFileAuthentication can also be used to specify kubeconfig path manually
auth = TokenAuthentication(
    token = "sha256~v3mNsdN7WLXI0ZHL90OfoOGCiJg6JB5J_BBqRYx_Dwg",
    server = "https://api.aisrhods-dims.yani.p1.openshiftapps.com:6443",
    skip_tls=False
)
auth.login()

'Logged into https://api.aisrhods-dims.yani.p1.openshiftapps.com:6443'

In [11]:
cluster = Cluster(ClusterConfiguration(
    name='stab-diff-model',
    namespace='default',
    num_workers=2,
    min_cpus=8,
    max_cpus=16,
    min_memory=4,
    max_memory=64,
    num_gpus=1,
    instascale=True, #<---instascale enabled
    machine_types=["m5.xlarge", "g5.4xlarge"],
))

Written to: stab-diff-model.yaml


In [12]:
cluster.up()

In [13]:
# cluster.down()

In [33]:
cluster.details()

                      🚀 CodeFlare Cluster Details 🚀                     
                                                                          
 ╭──────────────────────────────────────────────────────────────────────╮ 
 │   Name                                                               │ 
 │   stab-diff-model                                        Active ✅   │ 
 │                                                                      │ 
 │   URI: ray://stab-diff-model-head-svc.default.svc:10001              │ 
 │                                                                      │ 
 │   ]8;id=563716;http://ray-dashboard-stab-diff-model-default.apps.aisrhods-dims.yani.p1.openshiftapps.com\Dashboard🔗]8;;\                                                        │ 
 │                                                                      │ 
 │                       Cluster Resources                              │ 
 │   ╭── Workers ──╮  ╭───────── Worker specs(each) ─────────╮          │ 
 │   │  # Workers  │  │  Memory      CPU         GPU         │          │ 
 │   │             │  │                                      │          │ 
 │   │  2          │  │  4~64        8           1           │          │ 
 │   │             │  │                                      │          │ 
 │   ╰─────────────╯  ╰──────────────────────────────────────╯          │ 
 ╰──────────────────────────────────────────────────────────────────────╯

RayCluster(name='stab-diff-model', status=<RayClusterStatus.READY: 'ready'>, workers=2, worker_mem_min=4, worker_mem_max=64, worker_cpu=8, worker_gpu=1, namespace='default', dashboard='http://ray-dashboard-stab-diff-model-default.apps.aisrhods-dims.yani.p1.openshiftapps.com')

In [16]:
cluster.status()

╭──────────────────────────────────────────╮
│        🚀 Cluster Queue Status 🚀        │
│ +-----------------+--------------------+ │
│ | Name            | Status             | │
│ +=================+====================+ │
│ | stab-diff-model | running (starting) | │
│ |                 |                    | │
│ +-----------------+--------------------+ │
╰──────────────────────────────────────────╯

(<CodeFlareClusterStatus.STARTING: 2>, False)

In [3]:
import codeflare_sdk as cf

In [4]:
cf.cluster.cluster.list_all_clusters('default')

AttributeError: module 'codeflare_sdk' has no attribute 'cluster'

## Step 0: Prep

In [17]:
!pwd

/opt/app-root/src/teddy-demo


In [51]:
# THIS IS ORIGINAL

# TODO: If running on multiple nodes, change this path to a shared directory (ex: NFS)
!export DATA_PREFIX="/opt/app-root/src/teddy-demo"
!export ORIG_MODEL_NAME="CompVis/stable-diffusion-v1-4"
!export ORIG_MODEL_HASH="b95be7d6f134c3a9e62ee616f310733567f069ce"
!export ORIG_MODEL_DIR="$DATA_PREFIX/model-orig"
!export ORIG_MODEL_PATH="$ORIG_MODEL_DIR/models--${ORIG_MODEL_NAME/\//--}/snapshots/$ORIG_MODEL_HASH"
!export TUNED_MODEL_DIR="$DATA_PREFIX/model-tuned"
!export IMAGES_REG_DIR="$DATA_PREFIX/images-reg"
!export IMAGES_OWN_DIR="$DATA_PREFIX/images-own"
!export IMAGES_NEW_DIR="$DATA_PREFIX/images-new"
# TODO: Add more worker nodes and increase NUM_WORKERS for more data-parallelism
!export NUM_WORKERS=2

# !mkdir -p 
!echo $ORIG_MODEL_DIR 
# $TUNED_MODEL_DIR $IMAGES_REG_DIR $IMAGES_OWN_DIR $IMAGES_NEW_DIR
!mkdir -p $ORIG_MODEL_DIR $TUNED_MODEL_DIR $IMAGES_REG_DIR $IMAGES_OWN_DIR $IMAGES_NEW_DIR

/model-orig
mkdir: cannot create directory ‘/model-orig’: Permission denied
mkdir: cannot create directory ‘/model-tuned’: Permission denied
mkdir: cannot create directory ‘/images-reg’: Permission denied
mkdir: cannot create directory ‘/images-own’: Permission denied
mkdir: cannot create directory ‘/images-new’: Permission denied


In [19]:
import os

In [53]:
# THIS IS ORIGINAL

# TODO: If running on multiple nodes, change this path to a shared directory (ex: NFS)
DATA_PREFIX="/opt/app-root/src/teddy-demo"

# DATA_PREFIX="/opt/app-root/src/text-to-image-demo/dim-dreambooth"
ORIG_MODEL_NAME="CompVis/stable-diffusion-v1-4"
ORIG_MODEL_HASH="b95be7d6f134c3a9e62ee616f310733567f069ce"
ORIG_MODEL_DIR="model-orig"
ORIG_MODEL_PATH="$ORIG_MODEL_DIR/models--${ORIG_MODEL_NAME/\//--}/snapshots/$ORIG_MODEL_HASH"
TUNED_MODEL_DIR="model-tuned"
IMAGES_REG_DIR="images-reg"
IMAGES_OWN_DIR="images-own"
IMAGES_NEW_DIR="images-new"
# TODO: Add more worker nodes and increase NUM_WORKERS for more data-parallelism
!export NUM_WORKERS=2

os.makedirs(ORIG_MODEL_DIR, exist_ok=True)
os.makedirs(TUNED_MODEL_DIR, exist_ok=True)
os.makedirs(IMAGES_REG_DIR, exist_ok=True)
os.makedirs(IMAGES_OWN_DIR, exist_ok=True)
os.makedirs(IMAGES_NEW_DIR, exist_ok=True)

In [33]:
# !export DATA_PREFIX="/opt/app-root/src/text-to-image-demo/dim-dreambooth"
# !export ORIG_MODEL_DIR="$DATA_PREFIX/model-orig"
# !export ORIG_MODEL_NAME="CompVis/stable-diffusion-v1-4"
# !export ORIG_MODEL_HASH="b95be7d6f134c3a9e62ee616f310733567f069ce"


In [54]:
!echo $DATA_PREFIX
!echo $ORIG_MODEL_NAME
!echo $ORIG_MODEL_HASH

/opt/app-root/src/teddy-demo
CompVis/stable-diffusion-v1-4
b95be7d6f134c3a9e62ee616f310733567f069ce


## Step 1: Download the pre-trained model

In [23]:
!pwd

/opt/app-root/src/teddy-demo


In [32]:
# !python 05_dreambooth_finetuning/dreambooth/cache_model.py --model_dir=$DATA_PREFIX/images-reg --model_name=$ORIG_MODEL_NAME --revision=$ORIG_MODEL_HASH

## Step 2: Supply images of your subject

In [56]:
  # Only uncomment one of the following:

  # Option 1: Use the dog dataset ---------
!export CLASS_NAME="dog"
# !python 05_dreambooth_finetuning/dreambooth/download_example_dataset.py ./images/dog
CLASS_NAME='dog'
  # export INSTANCE_DIR=./images/dog
  # ---------------------------------------

  # Option 2: Use the lego car dataset ----
 # !export CLASS_NAME="car"
 # !export INSTANCE_DIR=./images/lego-car
  # ---------------------------------------

  # Option 3: Use your own images ---------
  # export CLASS_NAME="<class-of-your-subject>"
  # export INSTANCE_DIR="/path/to/images/of/subject"
  # ---------------------------------------

  # Copy own images into IMAGES_OWN_DIR
# !cp -rf $INSTANCE_DIR/* "$IMAGES_OWN_DIR/"

## Step 3: Create the regularization images

In [55]:
!echo $CLASS_NAME

In [31]:
!pip install flags

  Preparing metadata (setup.py) ... done
  Created wheel for flags: filename=flags-0.0.1.2-py3-none-any.whl size=1698 sha256=3803f64bf6424a9f7380aebf4f6b56a5af600181899edcffd42107cfb5577489
  Stored in directory: /tmp/pip-ephem-wheel-cache-_3zwiw96/wheels/70/d5/c6/b10a3579bc3ccbcab5fb92dfbb8b16e075cd86dab671fce1a1
Successfully built flags

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [30]:
!pip install pyarrow


[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [57]:
!python 05_dreambooth_finetuning/dreambooth/generate.py \
    --model_dir=ORIG_MODEL_PATH \
    --output_dir=IMAGES_REG_DIR \
    --prompts=f"photo of a dog" \
    --num_samples_per_prompt=200 \
    --use_ray_data

2023-10-08 10:07:10,981	INFO packaging.py:518 -- Creating a file package for local directory '.'.
2023-10-08 10:07:10,987	WARNING packaging.py:393 -- File /opt/app-root/src/teddy-demo/images-reg/models--CompVis--stable-diffusion-v1-4/snapshots/b95be7d6f134c3a9e62ee616f310733567f069ce/safety_checker/model.fp16.safetensors is very large (579.85MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': ['/opt/app-root/src/teddy-demo/images-reg/models--CompVis--stable-diffusion-v1-4/snapshots/b95be7d6f134c3a9e62ee616f310733567f069ce/safety_checker/model.fp16.safetensors']})`
2023-10-08 10:07:12,422	WARNING packaging.py:393 -- File /opt/app-root/src/teddy-demo/images-reg/models--CompVis--stable-diffusion-v1-4/snapshots/b95be7d6f134c3a9e62ee616f310733567f069ce/safety_checker/pytorch_model.fp16.bin is very large (579.93MiB). Consider adding this file to the 'excludes' list to skip uploading it: `ray.init(..., runtime_env={'excludes': [

## Fine tune the model

In [5]:
# !pip install ray['train'] --upgrade --force-reinstall

In [ ]:
UNIQUE_TOKEN = 'unqtkn'
CLASS_NAME = 'dog'

In [62]:
!pip install gputil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=cddb6c5d69c444075c5c3090b592463eb4333c7c948c6bd36c768a52020403e7
  Stored in directory: /tmp/pip-ephem-wheel-cache-hmeq78fu/wheels/2b/b5/24/fbb56595c286984f7315ee31821d6121e1b9828436021a88b3
Successfully built gputil

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [14]:
!pip install ray['rllib'] --upgrade --force-reinstall
!pip install ray['Train'] --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 137.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 103.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 182.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 188.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 144.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.0/531.0 kB 181.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 136.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 150.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 180.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 158.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 187.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 159.3 MB/s eta 0:00:00a 0:00:01


In [13]:
!pip install ray['Air'] --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 146.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 179.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 137.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 177.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 125.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 147.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 147.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.0/531.0 kB 181.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 190.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 160.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 168.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 156.9 MB/s eta 0:00:00
 

In [21]:
!pip install xformers --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 171.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 166.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 167.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 165.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 185.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 171.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 165.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 166.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 172.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 152.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 162.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━

In [10]:
# !pip install pyarrow --upgrade --force-reinstall

In [9]:
# !pip install grpcio --upgrade --force-reinstall

In [18]:
!pip install ray['Data'] --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 169.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 738.9/738.9 kB 191.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 131.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.0/531.0 kB 175.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 146.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 162.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 165.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.3/83.3 kB 142.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 176.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 179.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 161.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.4/173.4 kB 174.9 MB/s eta 0:00:00
 

In [30]:
!pip install flags --upgrade --force-reinstall

  Preparing metadata (setup.py) ... done
  Created wheel for flags: filename=flags-0.0.1.2-py3-none-any.whl size=1698 sha256=4d3390a4476676972446cd7c52d31e3f8471d0f9694ee8df7f27ed955ed02883
  Stored in directory: /tmp/pip-ephem-wheel-cache-d2z0bfd5/wheels/70/d5/c6/b10a3579bc3ccbcab5fb92dfbb8b16e075cd86dab671fce1a1
Successfully built flags
  Attempting uninstall: flags
    Found existing installation: flags 0.0.1.2
    Uninstalling flags-0.0.1.2:
      Successfully uninstalled flags-0.0.1.2

[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


## Interactive

### Imports

In [31]:
from typing import Dict

import itertools
from diffusers import (
    AutoencoderKL,
    DDPMScheduler,
    DiffusionPipeline,
    UNet2DConditionModel,
)

# LoRA related imports begin ##
from diffusers.loaders import (
    LoraLoaderMixin,
    text_encoder_lora_state_dict,
)
from diffusers.models.attention_processor import (
    AttnAddedKVProcessor,
    AttnAddedKVProcessor2_0,
    LoRAAttnAddedKVProcessor,
    LoRAAttnProcessor,
    LoRAAttnProcessor2_0,
    SlicedAttnAddedKVProcessor,
)

# LoRA related imports end ##
from diffusers.utils.import_utils import is_xformers_available
from ray.train import ScalingConfig
from ray import train
from ray.train.torch import TorchTrainer
import torch
import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from transformers import CLIPTextModel

from flags import train_arguments
import ray as ray

LORA_RANK = 4
from dataset import collate, get_train_dataset


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:35                                                                                   │
│                                                                                                  │
│   32 from torch.nn.utils import clip_grad_norm_                                                  │
│   33 from transformers import CLIPTextModel                                                      │
│   34                                                                                             │
│ ❱ 35 from flags import train_arguments                                                           │
│   36 import ray as ray                                                                           │
│   37                                                                                             │
│   38 LORA_RANK = 4                                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'train_arguments' from 'flags' 
(/opt/app-root/lib64/python3.9/site-packages/flags.py)

In [24]:
def prior_preserving_loss(model_pred, target, weight):
    # Chunk the noise and model_pred into two parts and compute
    # the loss on each part separately.
    model_pred, model_pred_prior = torch.chunk(model_pred, 2, dim=0)
    target, target_prior = torch.chunk(target, 2, dim=0)

    # Compute instance loss
    loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")

    # Compute prior loss
    prior_loss = F.mse_loss(
        model_pred_prior.float(), target_prior.float(), reduction="mean"
    )

    # Add the prior loss to the instance loss.
    return loss + weight * prior_loss


In [25]:
def get_target(scheduler, noise, latents, timesteps):
    """Get the target for loss depending on the prediction type."""
    pred_type = scheduler.config.prediction_type
    if pred_type == "epsilon":
        return noise
    if pred_type == "v_prediction":
        return scheduler.get_velocity(latents, noise, timesteps)
    raise ValueError(f"Unknown prediction type {pred_type}")


In [26]:
def add_lora_layers(unet, text_encoder):
    """Add LoRA layers for unet and text encoder.

    `unet` and `text_encoder` will be modified in place.

    Returns:
        The LoRA parameters for unet and text encoder correspondingly.
    """
    unet_lora_attn_procs = {}
    unet_lora_parameters = []
    for name, attn_processor in unet.attn_processors.items():
        cross_attention_dim = (
            None
            if name.endswith("attn1.processor")
            else unet.config.cross_attention_dim
        )
        if name.startswith("mid_block"):
            hidden_size = unet.config.block_out_channels[-1]
        elif name.startswith("up_blocks"):
            block_id = int(name[len("up_blocks.")])
            hidden_size = list(reversed(unet.config.block_out_channels))[block_id]
        elif name.startswith("down_blocks"):
            block_id = int(name[len("down_blocks.")])
            hidden_size = unet.config.block_out_channels[block_id]

        if isinstance(
            attn_processor,
            (AttnAddedKVProcessor, SlicedAttnAddedKVProcessor, AttnAddedKVProcessor2_0),
        ):
            lora_attn_processor_class = LoRAAttnAddedKVProcessor
        else:
            lora_attn_processor_class = (
                LoRAAttnProcessor2_0
                if hasattr(F, "scaled_dot_product_attention")
                else LoRAAttnProcessor
            )

        module = lora_attn_processor_class(
            hidden_size=hidden_size,
            cross_attention_dim=cross_attention_dim,
            rank=LORA_RANK,
        )
        unet_lora_attn_procs[name] = module
        unet_lora_parameters.extend(module.parameters())

    unet.set_attn_processor(unet_lora_attn_procs)

    text_lora_parameters = LoraLoaderMixin._modify_text_encoder(
        text_encoder, dtype=torch.float32, rank=LORA_RANK
    )

    return unet_lora_parameters, text_lora_parameters


In [27]:
def load_models(config):
    """Load pre-trained Stable Diffusion models."""
    # Load all models in bfloat16 to save GRAM.
    # For models that are only used for inferencing,
    # full precision is also not required.
    dtype = torch.bfloat16

    text_encoder = CLIPTextModel.from_pretrained(
        args.model_dir,
        subfolder="text_encoder",
        torch_dtype=dtype,
    )

    noise_scheduler = DDPMScheduler.from_pretrained(
        config["model_dir"],
        subfolder="scheduler",
        torch_dtype=dtype,
    )

    # VAE is only used for inference, keeping weights in full precision is not required.
    vae = AutoencoderKL.from_pretrained(
        config["model_dir"],
        subfolder="vae",
        torch_dtype=dtype,
    )
    # We are not training VAE part of the model.
    vae.requires_grad_(False)

    # Convert unet to bf16 to save GRAM.
    unet = UNet2DConditionModel.from_pretrained(
        config["model_dir"],
        subfolder="unet",
        torch_dtype=dtype,
    )

    if is_xformers_available():
        unet.enable_xformers_memory_efficient_attention()

    if not config["use_lora"]:
        unet_trainable_parameters = unet.parameters()
        text_trainable_parameters = text_encoder.parameters()
    else:
        text_encoder.requires_grad_(False)
        unet.requires_grad_(False)
        unet_trainable_parameters, text_trainable_parameters = add_lora_layers(
            unet, text_encoder
        )

    text_encoder.train()
    unet.train()

    torch.cuda.empty_cache()

    return (
        text_encoder,
        noise_scheduler,
        vae,
        unet,
        unet_trainable_parameters,
        text_trainable_parameters,
    )

In [28]:
@ray.remote
def train_fn(config):

    # Load pre-trained models.
    (
        text_encoder,
        noise_scheduler,
        vae,
        unet,
        unet_trainable_parameters,
        text_trainable_parameters,
    ) = load_models(config)

    text_encoder = train.torch.prepare_model(text_encoder)
    unet = train.torch.prepare_model(unet)
    # manually move to device as `prepare_model` can't be used on
    # non-training models.
    vae = vae.to(train.torch.get_device())

    # Use the regular AdamW optimizer to work with bfloat16 weights.
    optimizer = torch.optim.AdamW(
        itertools.chain(unet_trainable_parameters, text_trainable_parameters),
        lr=config["lr"],
    )

    train_dataset = train.get_dataset_shard("train")

    # Train!
    num_train_epochs = config["num_epochs"]

    print(f"Running {num_train_epochs} epochs.")

    global_step = 0
    for _ in range(num_train_epochs):
        if global_step >= config["max_train_steps"]:
            print(f"Stopping training after reaching {global_step} steps...")
            break

        for _, batch in enumerate(
            train_dataset.iter_torch_batches(
                batch_size=config["train_batch_size"],
                device=train.torch.get_device(),
            )
        ):
            batch = collate(batch, torch.bfloat16)

            optimizer.zero_grad()

            # Convert images to latent space
            latents = vae.encode(batch["images"]).latent_dist.sample() * 0.18215

            # Sample noise that we'll add to the latents
            noise = torch.randn_like(latents)
            bsz = latents.shape[0]
            # Sample a random timestep for each image
            timesteps = torch.randint(
                0,
                noise_scheduler.config.num_train_timesteps,
                (bsz,),
                device=latents.device,
            )
            timesteps = timesteps.long()

            # Add noise to the latents according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # Get the text embedding for conditioning
            encoder_hidden_states = text_encoder(batch["prompt_ids"])[0]

            # Predict the noise residual.
            model_pred = unet(
                noisy_latents.to(train.torch.get_device()),
                timesteps.to(train.torch.get_device()),
                encoder_hidden_states.to(train.torch.get_device()),
            ).sample
            target = get_target(noise_scheduler, noise, latents, timesteps)

            loss = prior_preserving_loss(
                model_pred, target, config["prior_loss_weight"]
            )
            loss.backward()

            # Gradient clipping before optimizer stepping.
            clip_grad_norm_(
                itertools.chain(unet_trainable_parameters, text_trainable_parameters),
                config["max_grad_norm"],
            )

            optimizer.step()  # Step all optimizers.

            global_step += 1
            results = {
                "step": global_step,
                "loss": loss.detach().item(),
            }
            train.report(results)

            if global_step >= config["max_train_steps"]:
                break
    # END: Training loop

    # Create pipeline using the trained modules and save it.
    if train.get_context().get_world_rank() == 0:
        if not config["use_lora"]:
            pipeline = DiffusionPipeline.from_pretrained(
                config["model_dir"],
                text_encoder=text_encoder.module,
                unet=unet.module,
            )
            pipeline.save_pretrained(config["output_dir"])
        else:
            save_lora_weights(unet.module, text_encoder.module, config["output_dir"])


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱   1 @ray.remote                                                                                │
│     2 def train_fn(config):                                                                      │
│     3 │                                                                                          │
│     4 │   # Load pre-trained models.                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'ray' is not defined

In [ ]:
def unet_attn_processors_state_dict(unet) -> Dict[str, torch.tensor]:
    """
    Returns:
        a state dict containing just the attention processor parameters.
    """
    attn_processors = unet.attn_processors

    attn_processors_state_dict = {}

    for attn_processor_key, attn_processor in attn_processors.items():
        for parameter_key, parameter in attn_processor.state_dict().items():
            param_name = f"{attn_processor_key}.{parameter_key}"
            attn_processors_state_dict[param_name] = parameter
    return attn_processors_state_dict


In [ ]:
def save_lora_weights(unet, text_encoder, output_dir):
    unet_lora_layers_to_save = None
    text_encoder_lora_layers_to_save = None

    unet_lora_layers_to_save = unet_attn_processors_state_dict(unet)
    text_encoder_lora_layers_to_save = text_encoder_lora_state_dict(text_encoder)

    LoraLoaderMixin.save_lora_weights(
        output_dir,
        unet_lora_layers=unet_lora_layers_to_save,
        text_encoder_lora_layers=text_encoder_lora_layers_to_save,
    )

In [23]:
args = train_arguments()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 args = train_arguments()                                                                     │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'train_arguments' is not defined

In [ ]:
args = 'model_dir'=images-reg \
    --output_dir=model-tuned \
    --instance_images_dir=images-own \
    --instance_prompt="photo of unqtkn dog" \
    --class_images_dir=images-reg \
    --class_prompt="photo of a dog" \
    --train_batch_size=2 \
    --lr=5e-6 \
    --num_epochs=4 \
    --max_train_steps=200 \
    --num_workers=2

In [ ]:
args = train_arguments().parse_args()
print(args)
ray.data.DataContext.get_current().execution_options.verbose_progress = True
# Build training dataset.
train_dataset = get_train_dataset(args)

print(f"Loaded training dataset (size: {train_dataset.count()})")

    # Train with Ray Train TorchTrainer.
trainer = TorchTrainer(
    train_fn,
    train_loop_config=vars(args),
    scaling_config=ScalingConfig(
        use_gpu=True,
        num_workers=args.num_workers,
    ),
    datasets={
        "train": train_dataset,
    },
)
result = trainer.fit()

print(result)

In [103]:
!python 05_dreambooth_finetuning/dreambooth/train.py \
    --model_dir=images-reg \
    --output_dir=model-tuned \
    --instance_images_dir=images-own \
    --instance_prompt="photo of unqtkn dog" \
    --class_images_dir=images-reg \
    --class_prompt="photo of a dog" \
    --train_batch_size=2 \
    --lr=5e-6 \
    --num_epochs=4 \
    --max_train_steps=200 \
    --num_workers=2

Namespace(model_dir='images-reg', output_dir='model-tuned', use_lora=False, instance_images_dir='images-own', instance_prompt='photo of unqtkn dog', class_images_dir='images-reg', class_prompt='photo of a dog', train_batch_size=2, lr=5e-06, num_epochs=4, max_train_steps=200, prior_loss_weight=1.0, max_grad_norm=1.0, num_workers=2)
2023-10-08 12:01:42,797	INFO worker.py:1633 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2023-10-08 12:01:44,420	INFO read_api.py:406 -- To satisfy the requested parallelism of 28, each read task output is split into 2 smaller blocks.
2023-10-08 12:01:44,447	INFO read_api.py:406 -- To satisfy the requested parallelism of 28, each read task output is split into 28 smaller blocks.
2023-10-08 12:01:44,609	INFO streaming_executor.py:93 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[ReadImage->SplitBlocks(2)] -> TaskPoolMapOperator[MapBatches(<lambda>)->MapBatches(transform_image)->MapBatches(<lambda>)->MapBatches(process_

In [ ]:
cluster.details()

In [ ]:
from codeflare_sdk.job.jobs import DDPJobDefinition

In [ ]:
arg_list = [
    "--model_name_or_path", "gpt2",
    "--dataset_name", "wikitext",
    "--dataset_config_name", "wikitext-2-raw-v1",
    "--per_device_train_batch_size", "2",
    "--per_device_eval_batch_size", "2",
    "--do_train",
    "--do_eval",
    "--output_dir", "/tmp/test-clm",
    "--overwrite_output_dir"
]

In [ ]:
jobdef = DDPJobDefinition(
    name="gpttest",
    script="gpt_og.py",
    script_args=arg_list,
    scheduler_args={"requirements": "requirements_gpt.txt"}
)
job = jobdef.submit(cluster)

In [ ]:
job.status()

Retrieve raw log output at anytime with:

In [ ]:
job.logs()

View live updates for status, logs, and other information with:

In [ ]:
cluster.cluster_dashboard_uri()

In [ ]:
job.status()

In [ ]:
cluster.down()

In [ ]:
auth.logout()